In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-dataset/heart.csv


In [7]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv('/kaggle/input/heart-disease-dataset/heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


As we can see this data is already clean. Let unclean it by adding missing values/duplicate entries and changing datatypes so that we can preprocess it

In [20]:
file_path = '/kaggle/input/heart-disease-dataset/heart.csv'
data = pd.read_csv(file_path)

#Introducing missing values in random places
np.random.seed(0)
for col in data.columns:
    data.loc[data.sample(frac=0.1).index, col] = np.nan

#Changing data type to string
data['age'] = data['age'].astype(str)

#Adding duplicate entries
duplicates = data.sample(frac=0.1, random_state=1)
data = pd.concat([data, duplicates], ignore_index=True)

#Introducing outliers in some columns
data.loc[data.sample(frac=0.1, random_state=2).index, 'Cholesterol'] = 1000
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,Cholesterol
0,52.0,1.0,0.0,125.0,212.0,0.0,1.0,168.0,0.0,1.0,NaN,2.0,3.0,0.0,NaN
1,nan,1.0,0.0,140.0,203.0,1.0,0.0,155.0,1.0,3.1,0.0,0.0,3.0,0.0,NaN
2,nan,1.0,0.0,145.0,174.0,0.0,1.0,125.0,1.0,2.6,NaN,0.0,3.0,0.0,NaN
3,61.0,1.0,0.0,148.0,203.0,0.0,1.0,161.0,0.0,0.0,2.0,1.0,3.0,0.0,NaN
4,62.0,0.0,0.0,138.0,294.0,1.0,NaN,106.0,NaN,1.9,NaN,3.0,2.0,0.0,NaN


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          1127 non-null   object 
 1   sex          1011 non-null   float64
 2   cp           1013 non-null   float64
 3   trestbps     1013 non-null   float64
 4   chol         1011 non-null   float64
 5   fbs          1013 non-null   float64
 6   restecg      1011 non-null   float64
 7   thalach      1014 non-null   float64
 8   exang        1017 non-null   float64
 9   oldpeak      1012 non-null   float64
 10  slope        1017 non-null   float64
 11  ca           1013 non-null   float64
 12  thal         1021 non-null   float64
 13  target       1015 non-null   float64
 14  Cholesterol  113 non-null    float64
dtypes: float64(14), object(1)
memory usage: 132.2+ KB


Now we can see that the data is not clean and requires preprocessing

In [31]:
#converting datatype to required format
data['age'] = data['age'].astype(float)

In [33]:
#filling missing values with mean
data.fillna(data.mean(numeric_only=True).round(), inplace=True)

In [34]:
#removing duplicates
data.drop_duplicates(inplace=True)

In [35]:
#finding outliers
Q1 = data['Cholesterol'].quantile(0.25)
Q3 = data['Cholesterol'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data['Cholesterol'] = data['Cholesterol'].clip(lower=lower_bound, upper=upper_bound)

In [36]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,Cholesterol
0,52.0,1.0,0.0,125.0,212.0,0.0,1.0,168.0,0.0,1.0,1.0,2.0,3.0,0.0,1000.0
1,55.0,1.0,0.0,140.0,203.0,1.0,0.0,155.0,1.0,3.1,0.0,0.0,3.0,0.0,1000.0
2,55.0,1.0,0.0,145.0,174.0,0.0,1.0,125.0,1.0,2.6,1.0,0.0,3.0,0.0,1000.0
3,61.0,1.0,0.0,148.0,203.0,0.0,1.0,161.0,0.0,0.0,2.0,1.0,3.0,0.0,1000.0
4,62.0,0.0,0.0,138.0,294.0,1.0,1.0,106.0,0.0,1.9,1.0,3.0,2.0,0.0,1000.0


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 843 entries, 0 to 1024
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          843 non-null    float64
 1   sex          843 non-null    float64
 2   cp           843 non-null    float64
 3   trestbps     843 non-null    float64
 4   chol         843 non-null    float64
 5   fbs          843 non-null    float64
 6   restecg      843 non-null    float64
 7   thalach      843 non-null    float64
 8   exang        843 non-null    float64
 9   oldpeak      843 non-null    float64
 10  slope        843 non-null    float64
 11  ca           843 non-null    float64
 12  thal         843 non-null    float64
 13  target       843 non-null    float64
 14  Cholesterol  843 non-null    float64
dtypes: float64(15)
memory usage: 105.4 KB


After cleaning and preprocessing the data we have eliminated the rows with duplicate entries, changed data type to the relevant format and dealt with null values